# NLP !

For this homework we will perform several steps of the data cleaning and preparation of the data. We will perform some text data classification into spam or ham categories

In [570]:
#Import all the necessary packages
import pandas as pd
import numpy as np
from collections import defaultdict

# nltk Libraries
from nltk import stem
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
import nltk
from nltk.stem import PorterStemmer

# sklearn Libraries
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from collections import defaultdict

# Regulare Expressions import
import re
import string


# Imported library to deal with contractions
import sys  
# !{sys.executable} -m pip install contractions
import contractions

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

#### Import the data. Please rename columns to label and text.

In [571]:
# Importing Data sheet
df = pd.read_csv("spam.csv", encoding = "latin-1")

# Dropping Unneccessary Coumns
df.drop(labels=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

# Renaming the columns
df.rename({'v1':'label', "v2":"text"}, axis=1, inplace=True)

# Printing Head to show transformation
df.head()

,label,text
0,ham,Stuning even for the non-gamer:
1,ham,"Go until jurong point, crazy.. Available only ..."
2,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [572]:
df.shape

(5573, 2)

### 1) Let's normalize the text data, i.e. remove punctuation, make lower case, expand contractions, remove stopwords and so on...

In [573]:
# Step 1: Drop anything that is a blank row just incase it came in
df['text'].dropna(inplace=True)
df.shape
# Note, there were no na drops

(5573, 2)

In [574]:
# Step 2: Make everything lower case
df['text'] = [word.lower() for word in df['text']]
df.head()

,label,text
0,ham,stuning even for the non-gamer:
1,ham,"go until jurong point, crazy.. available only ..."
2,ham,ok lar... joking wif u oni...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


In [575]:
# Step 3: Expanding Contractions
df['text'] = [contractions.fix(word) for word in df['text']]
df.head()

,label,text
0,ham,stuning even for the non-gamer:
1,ham,"go until jurong point, crazy.. available only ..."
2,ham,ok lar... joking wif you oni...
3,ham,you dun say so early hor... you c already then...
4,ham,"nah i do not think he goes to usf, he lives ar..."


In [576]:
# Step 3: Tokenization of the corpus/df
df['text']= [word_tokenize(line) for line in df['text']]
df.head()

,label,text
0,ham,"[stuning, even, for, the, non-gamer, :]"
1,ham,"[go, until, jurong, point, ,, crazy, .., avail..."
2,ham,"[ok, lar, ..., joking, wif, you, oni, ...]"
3,ham,"[you, dun, say, so, early, hor, ..., you, c, a..."
4,ham,"[nah, i, do, not, think, he, goes, to, usf, ,,..."


In [577]:
# Step 4: Removing the stop words

# Creating List of Stop Words
stopwords = nltk.corpus.stopwords.words('english')

no_stop_words = []  # Creating a blank list to store new lines in
for line in df['text']:
    # Appending new line with no stop words to the list
    no_stop_words.append([word for word in line if word not in stopwords])

# reseting text to not have stop words
df['text'] = no_stop_words
df.head()

,label,text
0,ham,"[stuning, even, non-gamer, :]"
1,ham,"[go, jurong, point, ,, crazy, .., available, b..."
2,ham,"[ok, lar, ..., joking, wif, oni, ...]"
3,ham,"[dun, say, early, hor, ..., c, already, say, ...]"
4,ham,"[nah, think, goes, usf, ,, lives, around, though]"


In [578]:
# Step 5: Removing the punctation
    # Subset Creating List of punctuations
pattern = '[{}]'.format(re.escape(string.punctuation))
pattern

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\^_`\\{\\|\\}\\~]'

In [579]:
punc_regex = re.compile(pattern)

# Creating a list of no punctuation
no_punc = []
for line in df['text']:
    no_punc.append(list(filter(None , [punc_regex.sub('', word)  for word in line])))

df['text'] = no_punc
df.head()

,label,text
0,ham,"[stuning, even, nongamer]"
1,ham,"[go, jurong, point, crazy, available, bugis, n..."
2,ham,"[ok, lar, joking, wif, oni]"
3,ham,"[dun, say, early, hor, c, already, say]"
4,ham,"[nah, think, goes, usf, lives, around, though]"


In [580]:
#  Step 6: Mapping the labels over to 1/0
df.label = df.label.map({"ham":1, "spam":0})
df.head()

,label,text
0,1,"[stuning, even, nongamer]"
1,1,"[go, jurong, point, crazy, available, bugis, n..."
2,1,"[ok, lar, joking, wif, oni]"
3,1,"[dun, say, early, hor, c, already, say]"
4,1,"[nah, think, goes, usf, lives, around, though]"


### 2) Stem and Tokenize the messages

In [581]:
# Stemming
pstemmer = PorterStemmer()

# Creating a list of no punctuation
pstem = []
for line in df['text']:
    pstem.append([pstemmer.stem(word) for word in line])

df['text'] = pstem
df.head()

,label,text
0,1,"[stune, even, nongam]"
1,1,"[go, jurong, point, crazi, avail, bugi, n, gre..."
2,1,"[ok, lar, joke, wif, oni]"
3,1,"[dun, say, earli, hor, c, alreadi, say]"
4,1,"[nah, think, goe, usf, live, around, though]"


In [582]:
# Tokenization

# Mapping all the Nouns verbs adj, adverbs
word_map = defaultdict(lambda : wn.NOUN)
word_map['V'], word_map['J'], word_map['R'] = wn.VERB, wn.ADJ, wn.ADV

tokens = []
for line in enumerate(df['text']):
    # Starting the WordNetLemmatizer()
    lemmatize = WordNetLemmatizer()
    # Maping the word lemmatization and adding it back as a string list to the tokens list
    tokens.append(str([lemmatize.lemmatize(word, word_map[map[0]]) for word, map in pos_tag(line[1])]))

df['text'] = tokens
df.head()


,label,text
0,1,"['stune', 'even', 'nongam']"
1,1,"['go', 'jurong', 'point', 'crazi', 'avail', 'b..."
2,1,"['ok', 'lar', 'joke', 'wif', 'oni']"
3,1,"['dun', 'say', 'earli', 'hor', 'c', 'alreadi',..."
4,1,"['nah', 'think', 'goe', 'usf', 'live', 'around..."


### 3) Split your data into a training and testing set (fillin the #Indepedent variable and #Dependent variable below)

In [583]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df['text'],
    df['label'], 
    test_size = 0.1, random_state = 1)

X_test    

1447             ['cramp', 'stop', 'go', 'back', 'sleep']
2032    ['hi', 'sorri', 'miss', 'call', 'pl', 'call', ...
4432    ['wonder', 'okor', 'great', 'month', 'cherish'...
4643    ['hey', 'iouri', 'give', 'number', 'wyli', 'ry...
5275    ['plea', 'call', '08712402972', 'immedi', 'urg...
                              ...                        
4849    ['want', '2', 'get', 'lay', 'tonight', 'want',...
2506    ['hi', 'darlin', 'hope', 'nice', 'night', 'wis...
2128    ['lol', 'well', 'without', 'could', 'big', 'sa...
2463    ['go', 'death', 'go', 'leav', 'note', 'say', '...
5       ['even', 'brother', 'like', 'speak', 'treat', ...
Name: text, Length: 558, dtype: object

In [584]:
# Checking the shape of the tests
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5015,) (558,) (5015,) (558,)


In [585]:
# Encoding y variabels to arrays
Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

### 4) Transform the data using the TF-IDF method and fit the SVM model on the data.

In [586]:
#Fit the model :
# Get built-in TF-IDF method
vectorizer = TfidfVectorizer()
vectorizer.fit(df['text'])
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [587]:
#Fit in the transformed data to the SVM classifier. Not covered in class, so code is given for fitting 
#the data to the SVM classifier

#Creates an SVM classidier object 
svm = svm.SVC(C=1000)

#Fits the SVM model on the TF-IDF transformed data along with the train labels
svm.fit(X_train, y_train)

SVC(C=1000)

In [590]:
# Printing the accuracy of the test
print("Accuracy Score:", accuracy_score(svm.predict(X_test), y_test)*100)

Accuracy Score: 97.4910394265233
